## 聚类GAN：对抗生成网络的潜在空间聚类  
  
###  能够使用GAN进行聚类的动机   
  
  使用GAN进行聚类的动机：在潜在空间z中,从一种类型向另一种类型转变时。会引起高维样本的变化。这种变化包含了聚类的发生过程。我们的动机就是利用这种变化来完成聚类。（利用的方法就是在潜在空间里：想办法对还未向样本转为潜在变量进行聚类。）  
    
  这里的思想有点类似于：对样本需训练集进行提取特征，然后对特征进行聚类。  
    
  只不过ClusterGAN类似于这个过程的逆过程。
  
  
###  使用GAN进行聚类的思想    
  
   使用  （离散-连续）  的混合变量，从这个混合变量中抽样，进行GAN的训练（这个混合变量就在GAN空间的潜在变量）。这个离散成分代表了是属于哪个簇。连续变量成分代表样本。  
     
   以上是传统的GAN成分，完成这个GAN训练后，虽然得到了样本簇结构，但是没办法完成对新样本的分类。 而且对一些复杂数据集，以上结构还不够完美。  
     
   所以在训练上诉GAN的时候的要加一个附加的逆映射网络，这个逆映射网络不仅能够在训练的时候加强训练得到的簇结构，而且在训练完成之后，这个逆映射网络，可以单独拿出来用于新样本的分类，和特征提取。  
      
 <br>  
   
ClusterGAN网络结构：
      
  ![](img/4.PNG)    
      
   
    
  
### 使用GAN进行聚类有哪些难点    
   
   1、为什么原始GAN，的潜在空间不能做聚类？  
     
   2、高斯混合潜在变量，为什么不能做聚类？  
     
   3、如何 使用  （离散-连续） 变量完成潜在空间的聚类？  
     
   4、 如何 进行 （离散-连续） 变量 的采样？
     
   5、在 使用 （离散-连续） 变量 的GAN 完成训练后，怎么使用GAN的生成器进行聚类 ？   
     
   6、针对5的问题，有两种解决办法，进行比较?
     
   7、如何保留GAN的插值能力？
     
###  解决这些难点的方法    
  
  * 1、为什么原始GAN，的潜在空间不能做聚类？   
    
       原始潜在空间中，选择正态分布抽样潜在变量，也就是说所有类别的样本，的潜在变量都是平滑分布在潜在空间里，没有聚类结构。  
         
   <br>  
     
  *  2、高斯混合潜在变量，为什么不能做聚类？      
    
      使用高斯混合成分作为潜在变量的抽样分布，高斯混合成分倾向于拥挤。（不太懂）  
        
   <br>   
     
  * 3、如何 使用  （离散-连续） 变量完成潜在空间的聚类？   
    
      定义   （离散-连续） 变量：  $z=(z_n,z_c) \quad z_n \text{~}N(0,\sigma^2Id_n), \quad z_c=e_k ,\quad k\text{~}\mu\{1,2,……,k\}$  
        
      这里  $ z_n$ 使用正态分布，$\sigma^2$方差决定了，聚类簇的紧凑性。$z_c=e_k$使用的是one-hot向量，one-hot个数是聚类簇的个数。所以这个成分决定了要生成的样本是属于哪个类别的。  
        
   <br>  
     
  * 4、 如何 进行 （离散-连续） 变量 的采样？  
     
     定义好了 （离散-连续） 变量，那么该如何进行抽样。 这里抽样的关键是，如何使正态分布和one-hot向量，相互配合，既能完成样本的生成又能实现分类。  
       
    聚类的关键是：one-hot向量（簇类别信息）能够为GAN训练提供充分的信息，让GAN的生成器，只从one-hot提供的对应类别中产生样本。  
      
    实现这个关键要求的方法是：让潜在变量的每个维度  $z_n \text{~}N(0,\sigma^2Id_n)$的$z_n$取值远远小于1就可以了。例如让$\sigma=0.10$时，潜在变量每个维度有极大概率为$z_{n,j} \in \{-0.6,0.6\}\;<<1.0$  
      
    (簇内结构更紧密，簇间结构更远离)  
      
   <br>  
     
  * 5、 在 使用 （离散-连续） 变量 的GAN 完成训练后，怎么使用GAN的生成器进行聚类 ？     
    
    完成对使用（离散-连续） 变量的GAN的训练之后。我们得到的是一个能够生成指定类别样本的生成器。此时还没有做到对已有的训练集样本进行聚类.  
      
    (也就是说到目前为止：完成了生成样本的聚类。（有了生成指定类别样本的生成器），但是这些不能用于对真实训练集中的样本进行聚类)   
       
     解决办法有两个：  
       
       * 1、基于编码的改进反向传播算法  
         
         （单独拿出训练后的生成器G,用G求出样本$x_1$与G(Z)的最小误差，$||G(z)-X_1||_2^2$，这时参数是Z,求出最优的$Z^*$,也就求出$X_1$是属于哪个类的了）  
         
           完成了GAN的训练的后，我们得到生成器G,把生成器G单独拿出来用这个基于编码的改进反向传播算法来对训练集的样本进行分类，现在固定生成器G的参数不变。把隐变量z视为参数，对隐变量z进行最优化，得到$z^*$，这样不同类别的样本，在经过优化后，将得到属于各个类别的最优$Z^*$完成聚类。   
             
        <br>  
          
       * 2、联合训练  
         
            不再是像方法一那样，训练完GAN之后，把生成器单独拿出来用改进的反向传播算法来完成聚类。  
              
            方法二：的思想是把这个编码器直接联合进GAN网络里，一起训练。（这个编码器的输入是生成的样本，输出是样本的潜在向量，和样本簇信息的one-hot向量） 
    
      
  
  <br>  
  
   *  7、如何保留GAN的插值能力？  
     
       ClusterGAN的插值能力：GAN的插值能力是指生成样本时从一个类别平稳的过渡到另一个类别。 ClusterGAN的插值能力的实现：从一个one-hot向量，向另一个one-hot向量过渡。就可以实现插值能力。$z=(z_n \;,\mu z_c^{(1)}+(1-\mu  )z_c^{(2)}\;) \quad \mu \in [0,1]$
  
### 算法流程  
  
算法一;:  
 * 1、基于编码的改进反向传播算法    
     
     一般求最优潜在向量的方法为：  
     ![](img/8.PNG)  
       
     这个问题是1、非凸，2、最开始使用不同初始化得到的生成器G，会有不同的z嵌入空间  
       
     解决方法：1、K重启，2、z裁剪  
       
     <br>  
       
     算法思想：k重启，裁剪，只对$z_n$惩罚，不对$z_c$惩罚。  
       
 $\qquad\qquad\qquad\quad$关键思想就是：K重启： 每一次重启，只采样一个one-hot($z_c^k$),就保持$z_c$不变。只对$z_n$进行优化，就可以求出这一次重启时样本x属于
                                       这个类别的最小损失  
         
 $\qquad\qquad\qquad\qquad\quad$K次重启，求可以求得样本x最终属于哪个类别时，损失最小
   
 ![](img/7.PNG)
   
     
 <br>  
   
 <br>  
   
 <br>  
   
算法二：  
  
 * 2、联合训练   
   
  ![](img/4.PNG)     
   
 <br>  
 
 <br>   
 
 目标函数：
    
  ![](img/3.PNG)    